Markov decision process is a mathematical tool for modelling stochastic control problems. Another way to look at them is that they can be used for 
sequential decision making. Sidenote: There is a [great lecture series on YouTube](https://www.youtube.com/watch?v=2pWv7GOvuf0&list=PLqYmG7hTraZDM-OYHWgPebj2MfCFzFObQ) by David Silver which explains MDPs and how they are used in reinforcement learning.
<br>
A key assumption is that all the states follow the *Markov* property which basically states that the future is independent of the past given the present.

## Defining MDPs
An MDP is a 4-tuple *M* : 
*M* = (*S* ,*A* ,*P<sub>a</sub>* ,*R<sub>a</sub>*) <br>
* *S* is the state space
* *A* is the action space
* *P<sub>a</sub>(*s* ,*s'*)* is the probability of going from state *s* to *s'*
* *R<sub>a</sub>(*s* ,*s'*)* is the reward for going form state *s* to *s'*

Sometimes a discount factor is added to the problem definition to model the variance associated with time horizons. Basically &gamma; models how much we care
about short-term rewards (&gamma; = 0) and long term rewards (&gamma; =1)

## MDPs for inventory control
[Adapated from 6.246 Reinforcement Learning: Foundations and Methods — Lec3 — 1](https://web.mit.edu/6.246/www/notes/L3-notes.pdf)<br>
A common question in supply chain management is when and by how much to replenish the shelves in a store. Replenishment can be thought of as an stochatic control problem. <br>
<img src="\assets\warehouse manager.jpg" width="200" height="200" /><br>

Let's say that you are a warehouse manager looking after just one SKU (stock keeping unit).You review the inventory at the end of each month and decide how much inventory to order.
There is a cost associated with maintaining some inventory in the warehouse *h(s)*, with ordering inventory *C(a)*. Selling the items earns you an income of *f(q)*. If the demand is greater than the available inventory then the customer leaves your store to never come back i.e there are no backorders. Only constraint that we are considering now is maximum capacity of the warehouse *M*. <br>

**State space** - *s* ∈ *S* = {0, 1, ...., M} which is the number of goods we can store in our warehouse; constrained by maximum capacity of the warehouse <br>
**Action space** - For a state *s*, *a* ∈ *A(s)* = {0, 1, ..., M − s}. Action space depends on the current state as we cannot order more than the capacity of the warehouse. Basically the only action the warehouse manager can take is choosing how much to order <br>
**Reward** - *r<sub>t</sub>* = - *C(a<sub>t</sub>)* - *h(s<sub>t</sub> + a<sub>t</sub>)*  + *f([s<sub>t</sub> + a<sub>t</sub> - s<sub>t - 1</sub>]<sup>+</sup>)*  <br>

### A simple inventory example
Let us assume that we are responsible to controlling the inventory for a particular "Widget". We review the inventory every day. Every day the  demand for our particular "widget" is a random variable that follows the poisson distribution. 

In [8]:
from abc import ABC, abstractmethod

class Distribution(ABC):
    @abstractmethod
    def sample(self):
        pass

import random
class Die(Distribution):
        def __init__(self,sides):
             self.sides = sides

        def __repr__(self):
            return f"Die(sides = {self.sides})"

        def sample(self):
          return random.randint(1,self.sides)



six_sided = Die(6)
def roll_dice():
    return six_sided.sample() + six_sided.sample()         

print(Die(6))           

Die(sides = 6)


In [1]:
from __future__ import annotations

from abc import ABC, abstractmethod
from collections import Counter, defaultdict
from dataclasses import dataclass
import numpy as np
import random
from typing import (Callable, Dict, Generic, Iterator, Iterable, Mapping, Optional, Sequence, Tuple, TypeVar)

A = TypeVar('A')
B = TypeVar('B')

class Distribution(ABC, Generic[A]):
    """A probability distribution that we can sample"""
    @abstractmethod
    def sample(self) -> A:
        """Return a random sample from this distribution."""
        pass
    def sample_n(self,n:int) -> Sequence[A]:
        """Return n samples from this distribution"""
        return [self.sample() for _ in range(n)]
    @abstractmethod
    def expectation(self, f:Callable[[A],float]) -> float:
        """Return the expecation of f(x) where X is the random variable for the distribution and f is an arbitrary function from X to float"""
        pass
    def map(self,f:Callable[[A],B]) -> Distribution[B]:
        """Apply a function to the outcome of this distribution."""
        return SampledDistribution(lambda: f(self.sample()))
    def apply(self,f:Callable[[A],Distribution[B]]) -> Distribution[B]:
        """Apply a function that returns a distribution to the outcomes of this distribution. This lets us express *dependent random variables*"""
        def sample():
            a = self.sample()
            b_dist = f(a)
            return b_dist.sample()
        return SampledDistribution(sample)     


    class SampledDistribution(Distribution[A]):
        """Distribution defined by a function to sample it"""
        sampler: Callable[[],A]
        expectation_samples: int

        def __init__(self, sampler: Callable[[],A],expectation_samples: int = 10000):
            self.sampler = sampler
            self.expectation_samples = expectation_samples
        def sample(self) -> A:
            return self.sampler()           
        def expectation(self, f:Callable[[A],float]) -> float:
            """Return a sampled approximation of the expectation of f(x) for some f."""
            return sum(f(self.sample()) for _ in range(self.expectation_samples))/self.expectation_samples    
        


         



